# Python Assignment

The data are defined at the following cell:

In [1]:
import numpy as np

A = np.array([[0, 3, -1, 8],
     [-1, 11, -1, 3],
     [2, -1, 10, -1],
     [10, -1, 2, 0]], dtype=np.float64)
b = np.array([[15], [25], [-1], [6]], dtype=np.float64)

## Q1  
Write a program to test if the matrix A is strictly diagonally dominant.

In [2]:
## Q1

def is_strictly_diagonally_dominant(A):
    diag = np.diag(A)
    other = np.sum(A, 1) - diag
    
    if all([a > 0  for a in (np.abs(diag) - np.abs(other))]):
        print("A is strictly diagonally dominant")
        return True
    else:
        print("A is not strictly diagonally dominant")
        return False
        
is_strictly_diagonally_dominant(A)        

A is not strictly diagonally dominant


False

The result shows that A is not strictly diagonally dominant.
LU decomposition and Gauss-Seidel method require the input Matrix to be diagonally dominant.
We can swap the first row and the last row of A to achieve it.


In [3]:
ANew = A.copy()
ANew[0, :], ANew[3, :]= A[3, :], A[0, :]
bNew = np.array([b[3], b[1], b[2], b[0]])
is_strictly_diagonally_dominant(ANew)    


A is strictly diagonally dominant


True

## Q2 LU decomposition
In the following cell, we define 

LU_decomposition_Doolittle(A) 

LU_decomposition_Crout(A)

to perform LU decomposition using Doolittle and Crout method respectively.

Then solve_LUD(L, U, b) will first solve
Lz=b, 
then, solve
Ux = z



In [4]:
# Q2 

# Doolittle
def LU_decomposition_Doolittle(A):
    n=len(A[0])
    L = np.eye(n)
    U = np.zeros([n, n])
    U[0, :] = A[0, :]    
    L[:, 0] = A[:, 0]/A[0, 0]   
    
    for i in range(1, n):       
        for j in range(i, n):#U
            temp = np.dot(L[i, :i], U[:i, j])            
            U[i][j]=A[i][j] - temp
        for j in range(i+1, n):#L
            temp = np.dot(L[j, :i], U[:i, i])
            L[j][i] = (A[j][i] - temp)/U[i][i]           
            
    return L,U




In [5]:
# Doolittle
def LU_decomposition_Crout(A):
    n=len(A[0])
    L = np.zeros([n, n])
    U = np.eye(n)
    U[0, :] = A[0, :] / A[0, 0]
    L[:, 0] = A[:, 0]
    for i in range(1, n):   
        for j in range(i, n):#L
            temp = np.dot(L[j, :i], U[:i, i])
            L[j][i] = (A[j][i] - temp)  
        for j in range(i + 1, n):#U
            temp = np.dot(L[i, :i], U[:i, j])            
            U[i][j]=(A[i][j] - temp)/L[i, i]
        
    return L,U



In [6]:
def solve_LUD(L, U, b):
    n = b.shape[0]
    #z = np.dot(np.linalg.inv(L), b) # solve Lz = b
    z = np.zeros((n, 1))
    z[0, 0] = b[0, 0] / L[0, 0]
    for i in range(1, n):
        z[i, 0] = (b[i, 0] - np.dot(L[i, :i], z[:i,0])) / L[i, i]
    #print(z1, z)
    #x = np.dot(np.linalg.inv(U), z) # solve ux = z
    x = np.zeros((n, 1))
    x[n-1, 0] = z[n-1, 0]/U[n-1, n-1]
    for i in range(n-2, -1, -1):
        x[i, 0] = (z[i, 0] - np.dot(U[(i+1):, i], x[(i+1):,0])) / U[i, i]

    return x

We first perform LUD using Doolittle method:

In [8]:
L, U = LU_decomposition_Doolittle(ANew)
x1 = solve_LUD(L, U, bNew)
L, U, x1

(array([[ 1.        ,  0.        ,  0.        ,  0.        ],
        [-0.1       ,  1.        ,  0.        ,  0.        ],
        [ 0.2       , -0.0733945 ,  1.        ,  0.        ],
        [ 0.        ,  0.27522936, -0.08173077,  1.        ]]),
 array([[10.        , -1.        ,  2.        ,  0.        ],
        [ 0.        , 10.9       , -0.8       ,  3.        ],
        [ 0.        ,  0.        ,  9.5412844 , -0.77981651],
        [ 0.        ,  0.        ,  0.        ,  7.11057692]]),
 array([[ 0.6       ],
        [ 2.34862385],
        [-0.03365385],
        [ 1.11494253]]))

The diagonal of L is all 1 for Doolittle method.

Then we can perform Crout method:

In [11]:
L, U = LU_decomposition_Crout(ANew)
x2 = solve_LUD(L, U, bNew)
print("Difference of two method |x1-x2|:\n", np.abs(x1-x2))
L, U, x2

Difference of two method |x1-x2|
 [[0.00000000e+00]
 [0.00000000e+00]
 [2.77555756e-17]
 [0.00000000e+00]]


(array([[10.        ,  0.        ,  0.        ,  0.        ],
        [-1.        , 10.9       ,  0.        ,  0.        ],
        [ 2.        , -0.8       ,  9.5412844 ,  0.        ],
        [ 0.        ,  3.        , -0.77981651,  7.11057692]]),
 array([[ 1.        , -0.1       ,  0.2       ,  0.        ],
        [ 0.        ,  1.        , -0.0733945 ,  0.27522936],
        [ 0.        ,  0.        ,  1.        , -0.08173077],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.6       ],
        [ 2.34862385],
        [-0.03365385],
        [ 1.11494253]]))

For Crout method, the diagonal of U is all one. The solution for the two methods are exactly the same.

The error of the method is mainly introduced by the calculation of division, especially when the denominator is near zero.
The time complexity of LU Decomposition is O(N^3). Solving a eqaution system with triangle matrix takes O(n^2).

## Q3 Gauss-Seidel Method



In [15]:
## Q3 Gauss-Seidel Method

def GSIterate(A, b, x, omega = 1.0):
    n = A.shape[0]
    x1 = np.zeros((n,1))
    
    for i in range(n):
        item1 = np.dot(A[i, :i], x1[:i, 0]) 
        item2 = np.dot(A[i, (i+1):], x[(i+1):, 0])
        x1[i] = (b[i] - item1 - item2)/A[i, i]
    return (1-omega) *x + omega*x1

def GaussSeidelMethod(A, b, x0, tol = 1.0e-4):
    x1 = GSIterate(A, b, x0)
    i = 1
    print(i, x1.T)
    while np.linalg.norm(x1 - x0, ord=np.inf) >tol:
        x0 = x1
        x1 = GSIterate(A, b, x0)
        i += 1
        print(i, x1.T)
    
    return x1


Using \[0, 0, 0, 0\] as the initial guess, the results are

In [16]:
x0 = np.array([[0], [0], [0], [0]])
GaussSeidelMethod(ANew, bNew, x0)

1 [[0.6        2.32727273 0.01272727 1.00386364]]
2 [[0.83018182 2.07557438 0.04190744 1.10189804]]
3 [[0.79917595 2.04867175 0.05522179 1.11365082]]
4 [[0.79382282 2.0461902  0.05721954 1.11483112]]
5 [[0.79317511 2.04599103 0.05744719 1.11493426]]
6 [[0.79310966 2.04597764 0.05746926 1.11494204]]


array([[0.79310966],
       [2.04597764],
       [0.05746926],
       [1.11494204]])

Using \[1, 1, 1, 1\] as the initial guess, the results are

In [17]:
x0 = np.array([[1], [1], [1], [1]])
GaussSeidelMethod(ANew, bNew, x0)

1 [[0.5        2.13636364 0.11363636 1.08806818]]
2 [[0.79090909 2.05821281 0.05644628 1.11022598]]
3 [[0.79453202 2.04730003 0.0568462  1.11436826]]
4 [[0.79336076 2.0461002  0.05737469 1.11488426]]
5 [[0.79313508 2.045987   0.05746011 1.11493739]]
6 [[0.79310668 2.04597769 0.05747017 1.11494214]]


array([[0.79310668],
       [2.04597769],
       [0.05747017],
       [1.11494214]])

Each iteration of the Guass Seidel Method takes O(N^2). When N is large enough  and GS method can converge quickly, the GS method can be more efficient than LUD. The coverge speed depends on the accuracy. GS method can be more flexiable. 

## Q4 SOR

In [19]:
## Q4

def GaussSeidelMethod_SOR(A, b, x0, omega, tol = 1.0e-4):
    x1 = GSIterate(A, b, x0, omega)
    i = 1
    print(i, x1.T)
    while np.linalg.norm(x1 - x0, ord=np.inf) >tol:
        x0 = x1
        x1 = GSIterate(A, b, x0, omega)
        i += 1
        print(i, x1.T)
    
    return x1

x0 = np.array([[0], [0], [0], [0]])
GaussSeidelMethod_SOR(ANew, bNew, x0, 1.1)

1 [[0.66    2.56    0.014   1.10425]]
2 [[0.87252    1.999445   0.047908   1.11244662]]
3 [[0.78214719 2.0501486  0.0587078  1.11540775]]
4 [[0.79438591 2.04556093 0.05736152 1.11489734]]
5 [[0.79295358 2.04601924 0.05748165 1.11494674]]
6 [[0.79312079 2.04597278 0.05747022 1.11494211]]
7 [[0.79310148 2.04597743 0.05747137 1.11494257]]


array([[0.79310148],
       [2.04597743],
       [0.05747137],
       [1.11494257]])

In [20]:
x0 = np.array([[1], [1], [1], [1]])
GaussSeidelMethod_SOR(ANew, bNew, x0, 1.1)

1 [[0.45     2.25     0.025    1.096875]]
2 [[0.857      2.0304375  0.0533     1.11399844]]
3 [[0.78592213 2.04732509 0.05792229 1.11511838]]
4 [[0.79387064 2.04583901 0.05743537 1.11492729]]
5 [[0.79301944 2.04599113 0.05747467 1.11494391]]
6 [[0.79311265 2.0459756  0.05747091 1.11494239]]


array([[0.79311265],
       [2.0459756 ],
       [0.05747091],
       [1.11494239]])

The acceleration factor does not accelerate the convergence speed.